In [83]:
import json
import urllib2
import urllib
import xmltodict
import isbntools.app as app
import os
import string

In [3]:
key=''.join(open("key").read().split('\n'))

In [20]:
query="The Sicilian Bought Bride by Carol Marinelli"

isbn = app.isbn_from_words(query)

print("The ISBN of the most `spoken-about` book with this title is %s" % isbn)
print("")
print("... and the book is:")
print("")
print((app.meta(isbn)))


The ISBN of the most `spoken-about` book with this title is 9780263185263

... and the book is:

{'Publisher': u'Thorndike Press', 'Language': u'en', 'Title': u"The Sicilian's Bought Bride", 'Authors': [u'Carol Marinelli'], 'ISBN-13': u'9780263185263', 'Year': u'2005'}


In [ ]:
user_id="56756503"
frmt="json"

In [28]:

#urlstring="https://www.goodreads.com/book/isbn/"+isbn+"?key="+key
urlstring="https://www.goodreads.com/search/index.xml?key="+key+"&q="+query.replace(" ","+")
print urlstring
url=urllib2.urlopen(urlstring)
url

https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=The+Sicilian+Bought+Bride+by+Carol+Marinelli


<addinfourl at 139961930527592 whose fp = <socket._fileobject object at 0x7f4b6d288c50>>

In [29]:
data=url.read()
data

'<?xml version="1.0" encoding="UTF-8"?>\n<GoodreadsResponse>\n  <Request>\n    <authentication>true</authentication>\n      <key><![CDATA[nXDOLuXC1fR2t3N43Gxng]]></key>\n    <method><![CDATA[search_index]]></method>\n  </Request>\n  <search>\n  <query><![CDATA[The Sicilian Bought Bride by Carol Marinelli]]></query>\n    <results-start>1</results-start>\n    <results-end>2</results-end>\n    <total-results>2</total-results>\n    <source>Goodreads</source>\n    <query-time-seconds>0.15</query-time-seconds>\n    <results>\n        <work>\n  <id type="integer">7459141</id>\n  <books_count type="integer">17</books_count>\n  <ratings_count type="integer">68</ratings_count>\n  <text_reviews_count type="integer">3</text_reviews_count>\n  <original_publication_year type="integer">2004</original_publication_year>\n  <original_publication_month type="integer" nil="true"/>\n  <original_publication_day type="integer" nil="true"/>\n  <average_rating>3.34</average_rating>\n  <best_book type="Book">\n

In [30]:

data1=xmltodict.parse(data)


In [64]:
#meta=data1[u'GoodreadsResponse'][u'book']
meta=data1[u'GoodreadsResponse'][u'search'][u'results'][u'work'][0][u'best_book']
meta[u'author'][u'name']

u'Carol Marinelli'

In [67]:
img=str(meta[u'image_url'])
title=str(meta[u'title'])
author=str(meta[u'author'][u'name'])

In [68]:
urllib.urlretrieve(img,author+" - "+title+".jpg")

("Carol Marinelli - The Sicilian's Bought Bride.jpg",
 <httplib.HTTPMessage instance at 0x7f4b6d2b02d8>)

In [124]:
#query contains book name and author
# key is the goodread API key
#path is location where the cover image shall be stored
def getMeta(query,key,path):
    urlstring="https://www.goodreads.com/search/index.xml?key="+key+"&q="+query.replace(" ","+")
    print urlstring
    try:
        url=urllib2.urlopen(urlstring)
        data=xmltodict.parse(url.read())
        if len(data)>0:
            meta=data[u'GoodreadsResponse'][u'search'][u'results'][u'work'][0][u'best_book']
            dictt={}
            dictt['title']=str(meta[u'title'])
            dictt['author']=str(meta[u'author'][u'name'])
            dictt['image']=path+dictt['author']+" - "+dictt['title']+".jpg"
            img=str(meta[u'image_url'])
            urllib.urlretrieve(img,dictt['image'])
            return dictt
        else:
            return query+" Not found"
    except:
        return query+" Not found"


In [125]:
query=""
getMeta(query,key,'/home/some/')

https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=


' Not found'

In [126]:
ls =! find /home/some/Downloads/ -type f -iname "*.html"
len(ls)

6

In [127]:
string.ascii_letters+" "

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '

In [130]:
def getFileMeta(f,key,path):

    fp,fn=os.path.split(f)
    print(f,fp[fp.rindex('/')+1:])
    qr=''.join(m for m in fp[fp.rindex('/')+1:] if m in string.ascii_letters+" ")
    qr=' '.join(q for q in qr.split() if len(q)>3)
    tm=qr.split()
    qrr=' '.join(tm[:2]+tm[-2:])
    print qrr
    return getMeta(qr,key,path)

In [131]:
for f in ls:
    getFileMeta(f,key,"/home/some/")

('/home/some/Downloads/Carol Marinelli Books/Carol Marinelli - Italian Boss, Ruthless Revenge/italianbossruthlessrevenge.html', 'Carol Marinelli - Italian Boss, Ruthless Revenge')
Carol Marinelli Ruthless Revenge
https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=Carol+Marinelli+Italian+Boss+Ruthless+Revenge
("/home/some/Downloads/Carol Marinelli Books/Carol Marinelli - The Sicilian's Bought Bride/Carol Marinelli - The Sicilian's Bought Bride.HTML", "Carol Marinelli - The Sicilian's Bought Bride")
Carol Marinelli Bought Bride
https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=Carol+Marinelli+Sicilians+Bought+Bride
("/home/some/Downloads/Carol Marinelli Books/Carol Marinelli - The Billionaire's Contract Bride/fsd/Carol Marinelli - [Lusty Billionaires Bundle Bk1] - The Billionaire's Contract Bride.html", 'fsd')

https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=
('/home/some/Downloads/Carol Marinelli Books/Carol Marinelli - Eng

In [ ]:
https://images-na.ssl-images-amazon.com/images/I/41PRV6JH3WL.jpg